# Dataset Overview:

Name: House Sales Prediction in King County, USA (from Kaggle)

Description: This dataset contains details about house sales in King County, which includes columns such as:
- id: A unique identifier for each sale
- date: The date of the sale
- price: The sale price of the house (target variable)
- bedrooms: Number of bedrooms
- bathrooms: Number of bathrooms
- sqft_living: Square footage of the living space
- sqft_lot: Square footage of the lot
- floors: Number of floors
- waterfront: Whether the house is waterfront or not
- view: Quality of the view (scale 0-4)
- condition: Condition of the house (scale 1-5)
- grade: Overall grade of the house (scale 1-13)
- sqft_above: Square footage of the house excluding the basement
- sqft_basement: Square footage of the basement
- yr_built: Year the house was built
- yr_renovated: Year the house was renovated (if applicable)
- zipcode: Zip code where the house is located
- lat: Latitude of the house
- long: Longitude of the house
- sqft_living15: Living room area in the nearest 15 neighbors
- sqft_lot15: Lot size of the nearest 15 neighbors

ReLU(x)=max(0,x)

PROS: Solves the vanishing gradient problem, allowing networks to train deeper architectures.

CONS: Can cause "dead neurons" where ReLU units stop learning if they get stuck at zero.

For this dataset, ReLU would be more useful in hidden layers since it’s less likely to suffer from vanishing gradients in a deep neural network.